In [1]:
# loading libraries

In [ ]:
import numpy as np
import pandas as pd

# pytorch
import torch
import torch.nn as nn
import torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard